This is a demonstration of the use of GFDL generated intake_esm catalogs on CMIP data using a spec that is unified for GFDL and CMIP data
Reference: https://github.com/aradhakrishnanGFDL/canopy-cats

In [16]:
import intake,os
import intake_esm

In [5]:
print(intake_esm.__version__) 

2020.6.11


INSERT THE LOCATION OF YOUR DATA CATALOG IN THE CELL BELOW TO EXPLORE SOMETHING ELSE

In [2]:
col_url = "/home/a1r/github/CatalogBuilder/scripts/catalogcmip-3.json"

In [106]:
col = intake.open_esm_datastore(col_url)

Example search for a variable that potentially is served on two grids, and has multiple member_id

In [4]:
cat_grid = col.search(variable_id="tos", table_id="Oday")

Inspect cat_grid and see how many unique values are in each column. E.g. grid_label has 2 unique values, member_id has 3 unique values, 2 unique versions (CMIP data special!)

In [24]:
cat_grid 

,unique
activity_id,1
institution_id,1
source_id,1
experiment_id,5
frequency,0
modeling_realm,0
table_id,1
member_id,3
grid_label,2
variable_id,1


Ok, curious.... what are the grid_labels , member_ids, version_ids available? I don't want them all. 

In [5]:
cat_grid.df["grid_label"].unique()

array(['gr', 'gn'], dtype=object)

In [56]:
type(cat_grid)

intake_esm.core.esm_datastore

In [6]:
cat_grid.df["member_id"].unique()

array(['r1i1p1f1', 'r2i1p1f1', 'r3i1p1f1'], dtype=object)

In [58]:
cat_grid.df["version_id"].unique()


array(['v20190726', 'v20180701'], dtype=object)

In [ ]:
#Let me group results by experiments to see what I want

In [70]:
cat_grid.df.groupby("experiment_id")[["grid_label","version_id","variable_id", "table_id","member_id"]].nunique()

,grid_label,version_id,variable_id,table_id,member_id
experiment_id,,,,,
1pctCO2,2,1,1,1,1
abrupt-4xCO2,2,1,1,1,1
esm-hist,2,1,1,1,1
historical,2,2,1,1,3
piControl,2,1,1,1,1


In [72]:
cat = cat_grid.search(grid_label='gn',member_id='r1i1p1f1',experiment_id='1pctCO2')

In [73]:
#Lets examine our selections before we dmget!

In [74]:
cat.df.groupby("variable_id")[["experiment_id", "grid_label","version_id","variable_id", "table_id","member_id"]].nunique()

,experiment_id,grid_label,version_id,variable_id,table_id,member_id
variable_id,,,,,,
tos,1,1,1,1,1,1


Narrow-down the search for my analysis and dmget only the necessary data

In [ ]:
cat = cat_grid.search(experiment_id='historical',grid_label='gr', member_id='ri1p1f1')
#NOTE: you may look for multiple variables and a single xarray dataset object for each experiment will have as many data arrays as the number of 
#variables

In [ ]:
cat_grid_latest['version_id'].unique()

array(['v20190726'], dtype=object)

In [ ]:
cat.df.groupby("variable_id")[["experiment_id", "member_id","grid_label","version_id","variable_id", "table_id"]].nunique()

,experiment_id,member_id,grid_label,version_id,variable_id,table_id
variable_id,,,,,,
tos,1,1,1,1,1,1


In [75]:
cat.df["path"]

0     /uda/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/1pctCO2/r1...
1     /uda/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/1pctCO2/r1...
2     /uda/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/1pctCO2/r1...
3     /uda/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/1pctCO2/r1...
4     /uda/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/1pctCO2/r1...
5     /uda/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/1pctCO2/r1...
6     /uda/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/1pctCO2/r1...
7     /uda/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/1pctCO2/r1...
8     /uda/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/1pctCO2/r1...
9     /uda/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/1pctCO2/r1...
10    /uda/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/1pctCO2/r1...
11    /uda/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/1pctCO2/r1...
12    /uda/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/1pctCO2/r1...
13    /uda/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/1pctCO2/r1...
14    /uda/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/1pctCO2/r1...
Name: path, dtype: object

In [76]:
#for simple dmget usage, just use this !dmget {file}
#use following to wrap the dmget call for each path in the catalog
def dmgetmagic(x):
    cmd = 'dmget %s'% str(x) 
    return os.system(cmd)

#OR refer to importing dmget ,  https://github.com/aradhakrishnanGFDL/canopy-cats/tree/main/notebooks/dmget.py

In [77]:
dmstatus = cat.df["path"].apply(dmgetmagic)

In [79]:
dset_dict = cat.to_dataset_dict(cdf_kwargs={'chunks': {'time':5}, 'decode_times': False})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'source_id.experiment_id.member_id.table_id.grid_label'


what are my dataset names? 

In [81]:
dset_dict.keys() #if you have more than one CMIP experiment, pass the version_id or use latest_version function so that each unique dataset has the desired version. 

dict_keys(['GFDL-ESM4.1pctCO2.r1i1p1f1.Oday.gn'])

Here is our xarray dataset object to play with. 

In [87]:
ds = dset_dict['GFDL-ESM4.1pctCO2.r1i1p1f1.Oday.gn']

In [88]:
ds

<xarray.Dataset> Size: 91GB
Dimensions:    (time: 54750, y: 576, x: 720, vertex: 4, bnds: 2)
Coordinates:
    lat        (y, x) float32 2MB dask.array<chunksize=(576, 720), meta=np.ndarray>
    lat_bnds   (y, x, vertex) float32 7MB dask.array<chunksize=(288, 360, 2), meta=np.ndarray>
    lon        (y, x) float32 2MB dask.array<chunksize=(576, 720), meta=np.ndarray>
    lon_bnds   (y, x, vertex) float32 7MB dask.array<chunksize=(288, 360, 2), meta=np.ndarray>
  * bnds       (bnds) float64 16B 1.0 2.0
  * time       (time) float64 438kB 0.5 1.5 2.5 ... 5.475e+04 5.475e+04
    time_bnds  (time, bnds) float64 876kB dask.array<chunksize=(5, 2), meta=np.ndarray>
  * x          (x) float64 6kB -299.8 -299.2 -298.8 -298.2 ... 58.75 59.25 59.75
  * y          (y) float64 5kB -77.91 -77.72 -77.54 -77.36 ... 89.47 89.68 89.89
Dimensions without coordinates: vertex
Data variables:
    tos        (time, y, x) float32 91GB dask.array<chunksize=(5, 576, 720), meta=np.ndarray>
Attributes: (12/57)
    title:                            NOAA GFDL GFDL-ESM4 model output prepar...
    external_variables:               areacello
    history:                          File was processed by fremetar (GFDL an...
    table_id:                         Oday
    activity_id:                      CMIP
    branch_method:                    standard
    ...                               ...
    intake_esm_attrs:grid_label:      gn
    intake_esm_attrs:variable_id:     tos
    intake_esm_attrs:version_id:      v20180701
    intake_esm_attrs:grid_label.1:    gn
    intake_esm_attrs:_data_format_:   netcdf
    intake_esm_dataset_key:           GFDL-ESM4.1pctCO2.r1i1p1f1.Oday.gn

In [89]:
ds.tos 

<xarray.DataArray 'tos' (time: 54750, y: 576, x: 720)> Size: 91GB
dask.array<concatenate, shape=(54750, 576, 720), dtype=float32, chunksize=(5, 576, 720), chunktype=numpy.ndarray>
Coordinates:
    lat      (y, x) float32 2MB dask.array<chunksize=(576, 720), meta=np.ndarray>
    lon      (y, x) float32 2MB dask.array<chunksize=(576, 720), meta=np.ndarray>
  * time     (time) float64 438kB 0.5 1.5 2.5 ... 5.475e+04 5.475e+04 5.475e+04
  * x        (x) float64 6kB -299.8 -299.2 -298.8 -298.2 ... 58.75 59.25 59.75
  * y        (y) float64 5kB -77.91 -77.72 -77.54 -77.36 ... 89.47 89.68 89.89
Attributes:
    long_name:      Sea Surface Temperature
    units:          degC
    cell_methods:   area: mean where sea time: mean
    cell_measures:  area: areacello
    standard_name:  sea_surface_temperature
    original_name:  tos

#SECOND example 

In [91]:
cat2 = col.search(variable_id="dissocos",table_id="Omon") # ,grid_label='gr')

In [92]:
cat2

,unique
activity_id,1
institution_id,1
source_id,1
experiment_id,6
frequency,0
modeling_realm,0
table_id,1
member_id,1
grid_label,1
variable_id,1


In [94]:
cat2.df.groupby("experiment_id")[["grid_label","version_id","variable_id", "table_id","member_id"]].nunique()

,grid_label,version_id,variable_id,table_id,member_id
experiment_id,,,,,
1pctCO2,1,1,1,1,1
abrupt-4xCO2,1,1,1,1,1
esm-hist,1,1,1,1,1
esm-piControl,1,1,1,1,1
historical,1,2,1,1,1
piControl,1,1,1,1,1


Wait, historical has 2 versions published? lets get the latest_version 

In [157]:
import numpy as np
np_version = cat2.df["version_id"].unique() #.sort_values(ascending=False)
latest_version = np.sort(np_version)[-1]

In [161]:
cat_latest = cat2.search(experiment_id='historical',version_id=latest_version)

In [163]:
cat_latest.df.groupby("experiment_id")[["grid_label","version_id","variable_id", "table_id","member_id"]].nunique()

,grid_label,version_id,variable_id,table_id,member_id
experiment_id,,,,,
historical,1,1,1,1,1


In [167]:
dmstatus = cat_latest.df["path"].apply(dmgetmagic)

In [168]:
dset_dict = cat_latest.to_dataset_dict(cdf_kwargs={'chunks': {'time':5}, 'decode_times': False})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'source_id.experiment_id.member_id.table_id.grid_label'


In [173]:
for k in dset_dict.keys(): 
    print(k)

GFDL-ESM4.historical.r1i1p1f1.Omon.gr


In [174]:
dset_dict['GFDL-ESM4.historical.r1i1p1f1.Omon.gr']

<xarray.Dataset> Size: 513MB
Dimensions:    (time: 1980, lat: 180, lon: 360, bnds: 2)
Coordinates:
  * lat        (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
    lat_bnds   (lat, bnds) float64 3kB dask.array<chunksize=(180, 2), meta=np.ndarray>
  * lon        (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
    lon_bnds   (lon, bnds) float64 6kB dask.array<chunksize=(360, 2), meta=np.ndarray>
  * time       (time) float64 16kB 15.5 45.0 74.5 ... 6.018e+04 6.021e+04
    time_bnds  (time, bnds) float64 32kB dask.array<chunksize=(5, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    dissocos   (time, lat, lon) float32 513MB dask.array<chunksize=(5, 180, 360), meta=np.ndarray>
Attributes: (12/57)
    title:                            NOAA GFDL GFDL-ESM4 model output prepar...
    history:                          File was processed by fremetar (GFDL an...
    external_variables:               areacello
    table_id:                         Omon
    activity_id:                      CMIP
    branch_method:                    standard
    ...                               ...
    intake_esm_attrs:grid_label:      gr
    intake_esm_attrs:variable_id:     dissocos
    intake_esm_attrs:version_id:      v20190726
    intake_esm_attrs:grid_label.1:    gr
    intake_esm_attrs:_data_format_:   netcdf
    intake_esm_dataset_key:           GFDL-ESM4.historical.r1i1p1f1.Omon.gr